In [1]:
!pip install psycopg2

  Using cached https://files.pythonhosted.org/packages/5c/1c/6997288da181277a0c29bc39a5f9143ff20b8c99f2a7d059cfb55163e165/psycopg2-2.8.3.tar.gz
  Created wheel for psycopg2: filename=psycopg2-2.8.3-cp37-cp37m-linux_x86_64.whl size=450523 sha256=3a642ec51d10489439ed5e19632eec2af92d04f3ba4d9a6f70adea46d1ac10f8
  Stored in directory: /home/jovyan/.cache/pip/wheels/48/06/67/475967017d99b988421b87bf7ee5fad0dad789dc349561786b
Successfully built psycopg2


In [1]:
import pandas as pd
import psycopg2 as pg2
import psycopg2.extras as pgx

In [2]:
con = pg2.connect(host='this_postgres', user='postgres', database='postgres')

In [3]:
cur = con.cursor(cursor_factory=pgx.RealDictCursor)

In [4]:
cur.execute("""
BEGIN;
CREATE TABLE from_jupyter_test (_id INTEGER, name TEXT, list DOUBLE PRECISION[], vector BYTEA);
COMMIT;
""")

In [7]:
cur.execute("""
BEGIN;
INSERT INTO from_jupyter_test VALUES (1, 'spam', '{1,2,3,4,5}');
INSERT INTO from_jupyter_test VALUES (2, 'eggs', '{1,4,9,16,25}');
COMMIT;
""")

In [8]:
cur.execute("""
SELECT * FROM from_jupyter_test;
""")

In [9]:
pd.DataFrame(cur.fetchall())

,_id,name,list,vector
0,1,spam,"[1.0, 2.0, 3.0, 4.0, 5.0]",None
1,2,eggs,"[1.0, 4.0, 9.0, 16.0, 25.0]",None


In [10]:
con.close()

In [12]:
con = pg2.connect(host='this_postgres', user='postgres', database='postgres')

In [13]:
cur = con.cursor(cursor_factory=pgx.RealDictCursor)

In [24]:
cur.execute("""
SELECT list FROM from_jupyter_test;
""")

In [25]:
results = cur.fetchall()
results

[RealDictRow([('list', [1.0, 2.0, 3.0, 4.0, 5.0])]),
 RealDictRow([('list', [1.0, 4.0, 9.0, 16.0, 25.0])])]

In [22]:
import numpy as np

In [26]:
ary_1 = np.array(results[0]['list'])
ary_2 = np.array(results[1]['list'])

In [27]:
bin_ary_1 = pg2.Binary(ary_1)
bin_ary_2 = pg2.Binary(ary_2)

In [45]:
update_sql = """
BEGIN;
UPDATE from_jupyter_test
SET vector = {} 
WHERE _id = {};
COMMIT;
""".format(bin_ary_1, 1)
cur.execute(update_sql)

In [46]:
update_sql = """
BEGIN;
UPDATE from_jupyter_test
SET vector = {} 
WHERE _id = {};
COMMIT;
""".format(bin_ary_2, 2)
cur.execute(update_sql)

In [47]:
cur.execute("""SELECT * FROM from_jupyter_test;""")

In [48]:
df_jup = pd.DataFrame(cur.fetchall())

In [49]:
df_jup

,_id,name,list,vector
0,1,spam,"[1.0, 2.0, 3.0, 4.0, 5.0]","[b'\x00', b'\x00', b'\x00', b'\x00', b'\x00', ..."
1,2,eggs,"[1.0, 4.0, 9.0, 16.0, 25.0]","[b'\x00', b'\x00', b'\x00', b'\x00', b'\x00', ..."


In [50]:
df_jup.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 4 columns):
_id       2 non-null int64
name      2 non-null object
list      2 non-null object
vector    2 non-null object
dtypes: int64(1), object(3)
memory usage: 192.0+ bytes


In [51]:
cur.execute("""SELECT vector FROM from_jupyter_test;""")

In [52]:
result = cur.fetchone()
result

RealDictRow([('vector', <memory at 0x7f7751d61108>)])

In [53]:
np.frombuffer(result['vector'])

array([1., 2., 3., 4., 5.])

In [54]:
result = cur.fetchone()
result

RealDictRow([('vector', <memory at 0x7f7751d611c8>)])

In [55]:
np.frombuffer(result['vector'])

array([ 1.,  4.,  9., 16., 25.])